# You work for the online store Ice, which sells video games all over the world. User and expert reviews, genres, platforms (e.g. Xbox or PlayStation), and historical data on game sales are available from open sources. You need to identify patterns that determine whether a game succeeds or not. This will allow you to spot potential big winners and plan advertising campaigns.

In [176]:
# Loading all the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats as sp
import streamlit as st
import plotly_express as px

In [177]:
df = pd.read_csv('games.csv')

In [178]:
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [179]:
# columns names all lower case
df.columns = df.columns.str.lower()

#filter out rows where name is blank
df = df[df['name'].notna()]

#The year of release column does not follow the date time format so we convert to integer to have values we can work with
df['year_of_release'] = df['year_of_release'].replace([float('inf'), -float('inf')], float('nan'))
df['year_of_release'] = df['year_of_release'].fillna(0).astype(int)

#ignore na value from rating column
df['rating'] = df['rating'].astype('str', errors='ignore')

In [180]:
average_critic_score = df['critic_score'].mean()
sd_critic_score = df['critic_score'].std()

print("Average Critic Score:", average_critic_score)
print("Standard Deviation of Critic Score:", sd_critic_score)
#determine max value
max_critic = df['critic_score'].max()
print("Max Critic Score:", max_critic)

Average Critic Score: 68.96767850559173
Standard Deviation of Critic Score: 13.938164552843201
Max Critic Score: 98.0


#### If we were to change the missing values from the critic_socre column to the average it wouldn't effect the mean.  However changing about half of vlues to the mean would drastically effect the distribution.  So would set missing values to 99 so it won't overlap with our valid values.  

In [181]:
#check for value of 9.9
count_99 = (df['user_score'] == 9.9).sum()
display(f'There are {count_99} values already equal to 9.9')
#deal with tbd values
df['user_score'] = df['user_score'].replace('tbd', 9.9).astype(float)

'There are 0 values already equal to 9.9'

In [182]:
#multiply the user_score by 10 so we can compare to critic_score
df['user_score'] = (df['user_score'] * 10)
#replace empty vallue in na column with "99"
df.fillna(99, inplace=True)

In [183]:
# create column total sales
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']
# move new column following the columns it sums
df.insert(8, 'total_sales', df.pop('total_sales'))

In [200]:
df['average_score'] = (df['critic_score'] + df['user_score']) / 2

In [201]:
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
Index: 16713 entries, 0 to 16714
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int32  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   total_sales      16713 non-null  float64
 9   critic_score     16713 non-null  float64
 10  user_score       16713 non-null  float64
 11  rating           16713 non-null  object 
 12  average_score    16713 non-null  float64
dtypes: float64(8), int32(1), object(4)
memory usage: 1.7+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,total_sales,critic_score,user_score,rating,average_score
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,82.54,76.0,80.0,E,78.0
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,40.24,99.0,99.0,nan,99.0
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,35.52,82.0,83.0,E,82.5
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,32.77,80.0,80.0,E,80.0
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,31.38,99.0,99.0,nan,99.0
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,30.26,99.0,99.0,nan,99.0
6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,29.80,89.0,85.0,E,87.0
7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,28.91,58.0,66.0,E,62.0
8,New Super Mario Bros. Wii,Wii,2009,Platform,14.44,6.94,4.70,2.24,28.32,87.0,84.0,E,85.5
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,28.31,99.0,99.0,nan,99.0


### Conclusion  

# Title of data analysis 

In [191]:
# group by total sales and platform
total_sales_by_platform = df.groupby('platform')['total_sales'].sum().reset_index()

# create bar charts
fig0 = px.bar(total_sales_by_platform, x='platform', y='total_sales', title='Total Sales by Platform', labels={'platform': 'Platform', 'total_sales': 'Total Sales'}, color='total_sales', color_continuous_scale='Viridis')

# Show the plot
fig0.show()

In [185]:
# group by year and games released
games_per_year = df.groupby('year_of_release').size().reset_index(name='count')

# create bar charts
fig1 = px.bar(games_per_year, x='year_of_release', y='count', title='Games Released by Year', labels={'year_of_release': 'Year of Release', 'count': 'Games'}, color='count', color_continuous_scale='Blues')
fig1.update_xaxes(range=[1990, 2017])
fig1.update_traces(marker_line_color='black', marker_line_width=1.5)
fig1.show()

In [186]:
# group by platform and games release
games_per_platform = df['platform'].value_counts().reset_index()
games_per_platform.columns = ['platform', 'count']

# sort ascending
games_per_platform = games_per_platform.sort_values(by='count', ascending=True)
# create a bar graph
fig2 = px.bar(games_per_platform, x='platform', y='count', title='Games Released by Platform', labels={'platform': 'Platform', 'count': 'Games'}, color='count', color_continuous_scale='mint')
fig2.update_traces(marker_line_color='black', marker_line_width=1.5)
fig2.show()

In [187]:
# filter platform by PS2 and group by year
ps2_data = df[df['platform'] == 'PS2']
ps2_per_year = ps2_data.groupby('year_of_release').size().reset_index(name='count')

# create a bar graph
fig3 = px.bar(ps2_per_year, x='year_of_release', y='count', title='PS2 Games Released by Year', labels={'year_of_release': 'Year of Release', 'count': 'Games'}, color='count', color_continuous_scale='aggrnyl')
fig3.update_xaxes(range=[2000, 2017])
fig3.update_traces(marker_line_color='black', marker_line_width=1.5)

fig3.show()

In [188]:
# filter platform by PS3 and group by year
pc_data = df[df['platform'] == 'PC']
pc_per_year = pc_data.groupby('year_of_release').size().reset_index(name='count')

# create a bar graph
fig4 = px.bar(pc_per_year, x='year_of_release', y='count', title='PC Games Released by Year', labels={'year_of_release': 'Year of Release', 'count': 'Games'}, color='count', color_continuous_scale='aggrnyl')
fig4.update_xaxes(range=[2000, 2017])
fig4.update_traces(marker_line_color='black', marker_line_width=1.5)
fig4.show()

In [189]:
# filter platform by PS3 and group by year
xone_data = df[df['platform'] == 'XOne']
pc_per_year = xone_data.groupby('year_of_release').size().reset_index(name='count')

# create a bar graph
fig5 = px.bar(pc_per_year, x='year_of_release', y='count', title='XOne Games Released by Year', labels={'year_of_release': 'Year of Release', 'count': 'Games'}, color='count', color_continuous_scale='aggrnyl')
fig5.update_xaxes(range=[2000, 2017])
fig5.update_traces(marker_line_color='black', marker_line_width=1.5)
fig5.show()

In [190]:
# filter platform by PS3 and group by year
p123_data = df[df['platform'] == 'PS4']
pc_per_year = p123_data.groupby('year_of_release').size().reset_index(name='count')

# create a bar graph
fig6 = px.bar(pc_per_year, x='year_of_release', y='count', title='PS4 Games Released by Year', labels={'year_of_release': 'Year of Release', 'count': 'Games'}, color='count', color_continuous_scale='aggrnyl')
fig6.update_xaxes(range=[2000, 2017])
fig6.update_traces(marker_line_color='black', marker_line_width=1.5)
fig6.show()

In [215]:
# Create the new column for total sales
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

# Group by name and sum the total sales
total_sales_by_name = df.groupby('name')['total_sales'].sum().reset_index()

# Sort by total sales and take the top 20
top_20_sales_by_name = total_sales_by_name.sort_values(by='total_sales', ascending=False).head(20)

# Create a bar chart using Plotly Express
fig = px.bar(top_20_sales_by_name, x='name', y='total_sales', title='Top 20 Total Sales by Game Name', labels={'name': 'Game Name', 'total_sales': 'Total Sales'}, color='total_sales', color_continuous_scale='hot')

# Show the plot
fig.show()

In [213]:

# Create the new column for total sales
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

# Group by rating and sum the total sales
total_sales_by_rating = df.groupby('rating')['total_sales'].sum().reset_index()

# Create a bar chart using Plotly Express
fig = px.bar(total_sales_by_rating, x='rating', y='total_sales', title='Total Sales by Rating', labels={'rating': 'Rating', 'total_sales': 'Total Sales'}, color='total_sales', color_continuous_scale='jet')

# Show the plot
fig.show()


In [199]:
# Create the new column for total sales
df['total_sales'] = df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']

# Group by genre and sum the total sales
total_sales_by_genre = df.groupby('genre')['total_sales'].sum().reset_index()

# Create a bar chart using Plotly Express
fig = px.bar(total_sales_by_genre, x='genre', y='total_sales', title='Total Sales by Genre', labels={'genre': 'Genre', 'total_sales': 'Total Sales'}, color='total_sales', color_continuous_scale='Viridis')

# Show the plot
fig.show()

In [212]:
# Filter out scores 99 or above
filtered_df = df[(df['average_score'] < 99) & (df['total_sales'] > 5)]

# Group by name and calculate the average score
avg_score_by_name = filtered_df.groupby('name')['average_score'].mean().reset_index()

# Sort by average score and take the top 20
top_20_avg_score_by_name = avg_score_by_name.sort_values(by='average_score', ascending=False).head(20)

# Create a bar chart using Plotly Express
fig = px.bar(top_20_avg_score_by_name, x='name', y='average_score', title='Top 20 Games by Average Score (Below 99)', labels={'name': 'Game Name', 'average_score': 'Average Score'}, color='average_score', color_continuous_scale='Viridis')

# Show the plot
fig.show()